(under construction)

Adapted from [a notebook](https://nbviewer.jupyter.org/github/marksubbarao/pyWWT_AAS225/blob/master/SgrStream.ipynb) by Mark SubbaRao that uses files from [here](http://www.astro.virginia.edu/~srm4n/Sgr/data.html) and visualizes the tidal disruption of the Sagittarius Dwarf Galaxy described in [Law & Majewski 2010](https://iopscience.iop.org/article/10.1088/0004-637X/714/1/229).

In [ ]:
import time

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, Column
from astropy.time import Time, TimeDelta

import numpy as np

from astroquery.vizier import Vizier
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

from pywwt.jupyter import WWTJupyterWidget

In [ ]:
wwt = WWTJupyterWidget()
#wwt

In [ ]:
# import and manipulate a table with stellar stream data
stars = Table.read('https://raw.githubusercontent.com/WorldWideTelescope/pywwt-notebooks/137d6a398b4ba0b6de332cd4603e8e40c74bedcf/tutorials/data/SgrTriax_DYN.dat',
                   format='ascii.basic')
#stars = Table.read('SgrTriax_DYN.dat', format='ascii.basic')
stars.keep_columns(["ra","dec","dist","Pcol"])
stars['ra'].unit = u.deg
stars['dec'].unit = u.deg
stars['dist'] *= 1000
stars['dist'].unit = u.pc
#stars.rename_column('dist','distance')

# add a column of colors for each point for pywwt to read
stars['colormap'] = '#000000'
for i, val in enumerate(stars['Pcol']):
    if val == -1:
        stars['colormap'][i] = '#c0c0c0' # silver
    elif val == 0 or val == 1:
        stars['colormap'][i] = '#ffa500' # orange
    elif val == 2 or val == 3:
        stars['colormap'][i] = '#ff00ff' # magenta
    elif val == 4 or val == 5:
        stars['colormap'][i] = '#00ffff' # cyan
    elif val == 6 or val == 7:
        stars['colormap'][i] = '#228b22' # forest green

In [ ]:
stars

In [ ]:
# display the stellar streams in the viewer
star_ly = wwt.layers.add_data_layer(table=stars, frame='Sky',
                                    lon_att='ra', lat_att='dec',
                                    alt_att='dist', alt_type='distance',
                                    cmap_att='depth', size_scale=20)

star_ly.far_side_visible = True

In [ ]:
# import and manipulate a table containing Sgr Dwarf's orbital data
orbit = Table.read('https://raw.githubusercontent.com/WorldWideTelescope/pywwt-notebooks/137d6a398b4ba0b6de332cd4603e8e40c74bedcf/tutorials/data/SgrTriax_orbit.dat',
                   format='ascii.basic')
#orbit = Table.read('SgrTriax_orbit.dat', format='ascii.basic')
orbit.keep_columns(['time', 'ra', 'dec', 'dist'])
orbit['ra'].unit = u.deg
orbit['dec'].unit = u.deg
orbit['time'].unit = u.Gyr
orbit['dist'] *= 1000
#orbit.rename_column('dist', 'distance')

In [ ]:
# translate times from gigayears, which are too large for pywwt, ...
# to days -- so 1 day in the viewer translates to 1 gigayear real-time
base = Time('2019-01-01', format='iso').jd
orbit['wwt_times'] = base
orbit['wwt_times'] += orbit['time']

In [ ]:
orbit

In [ ]:
# display Sgr Dwarf in the viewer
orb_ly = wwt.layers.add_data_layer(table=orbit[:300], frame='Sky',
                                   lon_att='ra', lat_att='dec',
                                   alt_att='dist', alt_type='distance',
                                   size_scale=20)

orb_ly.far_side_visible = True